In [ ]:

from google.colab import files
uploaded = files.upload()


Saving Greece_fraud_data.csv to Greece_fraud_data.csv
Saving Ireland_fraud_data.csv to Ireland_fraud_data.csv
Saving Italy_fraud_data.csv to Italy_fraud_data.csv


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f" Using device: {device}")

 Using device: cuda


In [ ]:
# =============================
# 📦 Imports
# =============================
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# =============================
# ⚙️ Settings
# =============================
BATCH_SIZE = 64
EPOCHS = 2
ROUNDS = 5
LR = 0.001

# =============================
# 🧠 Model
# =============================
class FraudDetectionModel(nn.Module):
    def __init__(self, input_dim):
        super(FraudDetectionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x):
        return self.model(x)

# =============================
# 🔄 Federated Utils
# =============================
def load_and_preprocess_uploaded(uploaded_dict):
    # Read uploaded files into DataFrames
    raw_dfs = [pd.read_csv(fn) for fn in uploaded_dict.keys()]
    combined_df = pd.concat(raw_dfs)
    # print("Data: ",combined_df)

    # Drop unused columns

    drop_cols = ['Transaction_ID', 'Timestamp', 'Is_Cross_Border','Customer_Type', "Time_Since_Last_Txn", "Txn_Region",'Merchant_ID', 'Customer_ID','Risk_Score', 'Fraud_Type']
    combined_df.drop(columns=[c for c in drop_cols if c in combined_df.columns], inplace=True)

    # Label Encoding
    for col in combined_df.select_dtypes(include='object').columns:
        le = LabelEncoder()
        combined_df[col] = le.fit_transform(combined_df[col])

    # Scale numeric columns
    num_cols = combined_df.drop(columns=['Is_Fraud']).select_dtypes(include=['int64', 'float64']).columns
    scaler = StandardScaler()
    combined_df[num_cols] = scaler.fit_transform(combined_df[num_cols])

    # Split back into original datasets
    split_sizes = [len(pd.read_csv(fn)) for fn in uploaded_dict.keys()]
    datasets = []
    start = 0
    for size in split_sizes:
        end = start + size
        datasets.append(combined_df.iloc[start:end].reset_index(drop=True))
        start = end

    return datasets


In [ ]:


def split_data(df):
    train, test = train_test_split(df, test_size=0.2, stratify=df['Is_Fraud'], random_state=42)
    return train.reset_index(drop=True), test.reset_index(drop=True)

def prepare_dataset(df):
    X = torch.tensor(df.drop(columns=['Is_Fraud']).values, dtype=torch.float32)
    y = torch.tensor(df['Is_Fraud'].values, dtype=torch.long)
    return DataLoader(TensorDataset(X, y), batch_size=BATCH_SIZE, shuffle=True)

def train_client(model, loader, epochs, lr, device, global_model=None, mu=0.0):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # criterion = nn.CrossEntropyLoss()
     # Compute class weights
    all_labels = []
    for _, y in loader:
        all_labels.extend(y.numpy())
    fraud_count = sum(1 for l in all_labels if l == 1)
    nonfraud_count = sum(1 for l in all_labels if l == 0)
    total = fraud_count + nonfraud_count
    weight = torch.tensor([1.0, total / fraud_count], dtype=torch.float32).to(device)
    # weight = torch.tensor([1.0, 20.0], dtype=torch.float32).to(device)

    criterion = nn.CrossEntropyLoss(weight=weight)

    for _ in range(epochs):
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)

            # FedProx regularization
            if global_model and mu > 0:
                prox_term = sum(((p - p_global) ** 2).sum()
                                for p, p_global in zip(model.parameters(), global_model.parameters()))
                loss += (mu / 2) * prox_term

            loss.backward()
            optimizer.step()

def average_models(global_model, client_models):
    global_dict = global_model.state_dict()
    for key in global_dict:
        global_dict[key] = torch.stack([client.state_dict()[key].float() for client in client_models], 0).mean(0)
    global_model.load_state_dict(global_dict)

def evaluate_and_return_accuracy(model, loader, device, label=""):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            preds = model(X).argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(y.numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"\n📊 Evaluation on {label}:")
    print(f"Accuracy: {acc:.4f}")
    print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
    print("Classification Report:\n", classification_report(all_labels, all_preds))
    return acc

# =============================
# 📊 Run and Compare FedAvg vs FedProx
# =============================
def run_and_compare(uploaded_dict, mu_values=[0.0, 0.1]):
    results = {mu: [] for mu in mu_values}

    for mu in mu_values:
        print(f"\n==============================")
        print(f"🚀 Training with {'FedAvg' if mu == 0.0 else f'FedProx (mu={mu})'}")
        print(f"==============================")

        datasets = load_and_preprocess_uploaded(uploaded_dict)
        splits = [split_data(df) for df in datasets]

        train_loaders = [prepare_dataset(train) for train, _ in splits]
        test_loaders = [prepare_dataset(test) for _, test in splits]

        input_dim = datasets[0].shape[1] - 1
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        global_model = FraudDetectionModel(input_dim).to(device)

        for rnd in range(ROUNDS):
            print(f"🔁 Round {rnd+1}/{ROUNDS}")
            client_models = []
            for loader in train_loaders:
                client_model = FraudDetectionModel(input_dim).to(device)
                client_model.load_state_dict(global_model.state_dict())
                train_client(client_model, loader, EPOCHS, LR, device, global_model, mu)
                client_models.append(client_model)
            average_models(global_model, client_models)

        for i, test_loader in enumerate(test_loaders):
            acc = evaluate_and_return_accuracy(global_model, test_loader, device, label=f"Client {i+1}")
            results[mu].append(acc)

    # Print final comparison
    print("\n📈 Accuracy Comparison Table:")
    print("Client\tFedAvg\t\tFedProx")
    for i in range(len(results[0.0])):
        print(f"Client {i+1}\t{results[0.0][i]:.4f}\t\t{results[0.1][i]:.4f}")

# =============================
# 🏁 Run
# =============================
run_and_compare(uploaded_dict=uploaded, mu_values=[0.0, 0.1])


🚀 Training with FedAvg
🔁 Round 1/5
🔁 Round 2/5
🔁 Round 3/5
🔁 Round 4/5
🔁 Round 5/5

📊 Evaluation on Client 1:
Accuracy: 0.9774
Confusion Matrix:
 [[29296   608]
 [   69    27]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     29904
           1       0.04      0.28      0.07        96

    accuracy                           0.98     30000
   macro avg       0.52      0.63      0.53     30000
weighted avg       0.99      0.98      0.99     30000


📊 Evaluation on Client 2:
Accuracy: 0.9846
Confusion Matrix:
 [[29520   390]
 [   71    19]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     29910
           1       0.05      0.21      0.08        90

    accuracy                           0.98     30000
   macro avg       0.52      0.60      0.53     30000
weighted avg       0.99      0.98      0.99     30000


📊 Evaluation on Client 3:

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve

def plot_pr_roc_curves(model, data_loader, device, label=""):
    model.eval()
    y_true = []
    y_probs = []

    with torch.no_grad():
        for X, y in data_loader:
            X = X.to(device)
            logits = model(X)
            probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()  # Probabilities for class 1
            y_probs.extend(probs)
            y_true.extend(y.numpy())

    # ROC AUC
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = roc_auc_score(y_true, y_probs)

    # PR Curve
    precision, recall, _ = precision_recall_curve(y_true, y_probs)

    # Plot
    plt.figure(figsize=(12, 5))

    # ROC
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--", color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {label}")
    plt.legend()

    # Precision-Recall
    plt.subplot(1, 2, 2)
    plt.plot(recall, precision, label=f"PR Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve - {label}")
    plt.legend()

    plt.tight_layout()
    plt.show()
